# Neural Network

We'll now use a Neural Network to predict the players identity.

In [70]:
%matplotlib notebook

import pylab as plt
import numpy as np
import seaborn as sns; sns.set()

import keras
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam

from sklearn.decomposition import PCA

Let's use the feature data generated by the last year and train the NN with it. After we'll try our new features

In [100]:
data = np.genfromtxt('kate_data_julien_sarah.csv', delimiter=',')
np.random.shuffle(data)

In [101]:
training_ratio = 0.85
l = len(data)
X = data[:,:-1]
y = data[:,-1]
X_train = X[:int(l*training_ratio)]
X_test = X[int(l*training_ratio):]
y_train = y[:int(l*training_ratio)]/2
y_test = y[int(l*training_ratio):]/2

In [102]:
y_train = keras.utils.np_utils.to_categorical(y_train.astype(int))
y_test = keras.utils.np_utils.to_categorical(y_test.astype(int))

# Dimensionality reduction with PCA

I didn't understand this part... Just copied. I think they've manually applied the dimension reduction and created this "error". If so, we could do that directly using SciKit

In [103]:
mu = X_train.mean(axis=0)
U,s,V = np.linalg.svd(X_train - mu, full_matrices=False)
Zpca = np.dot(X_train - mu, V.transpose())

Rpca = np.dot(Zpca[:,:2], V[:2,:]) + mu    # reconstruction
err = np.sum((X_train-Rpca)**2)/Rpca.shape[0]/Rpca.shape[1]
print('PCA reconstruction error with 2 PCs: ' + str(round(err,3)));
print(max(Zpca[:,0]))
print(min(Zpca[:,0]))
print(max(Zpca[:,1]))
print(min(Zpca[:,1]))

print(np.argmax(Zpca[:,0]))
print(np.argmax(Zpca[:,1]))

PCA reconstruction error with 2 PCs: 6.255
2233.586102314126
-22.32930875280139
159.2231182347611
-105.72923985409766
966
966


# Building and training of a dnn

In [104]:
m = Sequential()
m.add(Dense(150,  activation='relu', input_shape=(105,)))
#m.add(Dense(150,  activation='relu')) 
m.add(Dense(150,  activation='relu'))
m.add(Dense(150,  activation='relu'))
m.add(Dense(50,  activation='relu'))
m.add(Dense(2,  activation='sigmoid'))
m.compile(loss='categorical_crossentropy', optimizer = Adam(), metrics=['accuracy'])

history = m.fit(X_train, y_train, batch_size=10, epochs=20, verbose=1, validation_data = (X_test, y_test))

Train on 1360 samples, validate on 240 samples
Epoch 1/20
1360/1360 [==============================] - 2s 1ms/step - loss: 0.6710 - acc: 0.5309 - val_loss: 0.6250 - val_acc: 0.5083
Epoch 2/20
1360/1360 [==============================] - 0s 241us/step - loss: 0.5842 - acc: 0.6206 - val_loss: 0.5755 - val_acc: 0.7625
Epoch 3/20
1360/1360 [==============================] - 0s 268us/step - loss: 0.4992 - acc: 0.7809 - val_loss: 0.3996 - val_acc: 0.7958
Epoch 4/20
1360/1360 [==============================] - 0s 275us/step - loss: 0.3872 - acc: 0.8044 - val_loss: 0.5051 - val_acc: 0.7708
Epoch 5/20
1360/1360 [==============================] - 0s 253us/step - loss: 0.3575 - acc: 0.8235 - val_loss: 0.4983 - val_acc: 0.7875
Epoch 6/20
1360/1360 [==============================] - 0s 254us/step - loss: 0.3461 - acc: 0.8324 - val_loss: 0.4909 - val_acc: 0.7792
Epoch 7/20
1360/1360 [==============================] - 0s 249us/step - loss: 0.3398 - acc: 0.8301 - val_loss: 0.4255 - val_acc: 0.8000
Epo

In [105]:
#history = m.fit(X_train, y_train, batch_size=300, epochs=10, verbose=1)

In [106]:
y_pred = m.predict(X_test)

In [107]:
accuracy = m.evaluate(X_test, y_test)[1]
print("Précision old features: %.2f" % accuracy)

240/240 [==============================] - 0s 66us/step
Précision old features: 0.82


Let's try with ours now

In [94]:
X = np.genfromtxt('features_wave_julian_sarah.csv', delimiter=',')
y = np.genfromtxt('output_wave_julian_sarah.csv', delimiter=',')

p = np.random.permutation(len(X))
X, y = X[p], y[p]

training_ratio = 0.85
l = len(y)
X_train = X[:int(l*training_ratio)]
X_test = X[int(l*training_ratio):]
y_train = y[:int(l*training_ratio)]/2
y_test = y[int(l*training_ratio):]/2

y_train = keras.utils.np_utils.to_categorical(y_train.astype(int))
y_test = keras.utils.np_utils.to_categorical(y_test.astype(int))

In our case we have 12 dimensions to features instead. Let's apply the NN directly without a PCA to compare later

In [95]:
m = Sequential()
m.add(Dense(15,  activation='relu', input_shape=(12,)))
#m.add(Dense(15,  activation='relu')) 
m.add(Dense(15,  activation='relu'))
m.add(Dense(15,  activation='relu'))
m.add(Dense(4,  activation='relu'))
m.add(Dense(2,  activation='sigmoid'))
m.compile(loss='categorical_crossentropy', optimizer = Adam(), metrics=['accuracy'])

history = m.fit(X_train, y_train, batch_size=10, epochs=30, verbose=1, validation_data = (X_test, y_test))

Train on 1360 samples, validate on 240 samples
Epoch 1/30
1360/1360 [==============================] - 2s 1ms/step - loss: 0.6732 - acc: 0.5015 - val_loss: 0.6555 - val_acc: 0.5083
Epoch 2/30
1360/1360 [==============================] - 0s 122us/step - loss: 0.6434 - acc: 0.5000 - val_loss: 0.6294 - val_acc: 0.5250
Epoch 3/30
1360/1360 [==============================] - 0s 126us/step - loss: 0.6062 - acc: 0.5059 - val_loss: 0.5735 - val_acc: 0.5125
Epoch 4/30
1360/1360 [==============================] - 0s 126us/step - loss: 0.5548 - acc: 0.5346 - val_loss: 0.5615 - val_acc: 0.6167
Epoch 5/30
1360/1360 [==============================] - 0s 131us/step - loss: 0.5432 - acc: 0.5779 - val_loss: 0.5540 - val_acc: 0.5458
Epoch 6/30
1360/1360 [==============================] - 0s 131us/step - loss: 0.5438 - acc: 0.5375 - val_loss: 0.5550 - val_acc: 0.5292
Epoch 7/30
1360/1360 [==============================] - 0s 139us/step - loss: 0.5356 - acc: 0.5390 - val_loss: 0.5508 - val_acc: 0.5750
Epo

In [96]:
accuracy = m.evaluate(X_test, y_test)[1]
print("Précision New features: %.2f" % accuracy)

240/240 [==============================] - 0s 33us/step
Précision New features: 0.86


# We got a precision of 86%, which is better than the 82% of the last year. But we would need to do an average to compare

Let's apply PCA now to reduce the dimension to 3

In [108]:
model_pca3 = PCA(n_components=3)

# On entraîne notre modèle (fit) sur les données
model_pca3.fit(X)

# On applique le résultat sur nos données :
X_reduced3 = model_pca3.transform(X)

In [109]:
training_ratio = 0.85
l = len(y)
X_train = X_reduced3[:int(l*training_ratio)]
X_test = X_reduced3[int(l*training_ratio):]
y_train = y[:int(l*training_ratio)]/2
y_test = y[int(l*training_ratio):]/2

y_train = keras.utils.np_utils.to_categorical(y_train.astype(int))
y_test = keras.utils.np_utils.to_categorical(y_test.astype(int))

In [111]:
m = Sequential()
m.add(Dense(20,  activation='relu', input_shape=(3,)))
#m.add(Dense(20,  activation='relu')) 
m.add(Dense(20,  activation='relu'))
m.add(Dense(20,  activation='relu'))
m.add(Dense(5,  activation='relu'))
m.add(Dense(2,  activation='sigmoid'))
m.compile(loss='categorical_crossentropy', optimizer = Adam(), metrics=['accuracy'])

history = m.fit(X_train, y_train, batch_size=10, epochs=20, verbose=1, validation_data = (X_test, y_test))

Train on 1360 samples, validate on 240 samples
Epoch 1/20
1360/1360 [==============================] - 2s 2ms/step - loss: 0.5657 - acc: 0.7338 - val_loss: 0.5751 - val_acc: 0.8083
Epoch 2/20
1360/1360 [==============================] - 0s 139us/step - loss: 0.4888 - acc: 0.7757 - val_loss: 0.5545 - val_acc: 0.8125
Epoch 3/20
1360/1360 [==============================] - 0s 136us/step - loss: 0.4778 - acc: 0.7809 - val_loss: 0.5340 - val_acc: 0.8125
Epoch 4/20
1360/1360 [==============================] - 0s 142us/step - loss: 0.4688 - acc: 0.7868 - val_loss: 0.5265 - val_acc: 0.8083
Epoch 5/20
1360/1360 [==============================] - 0s 134us/step - loss: 0.4692 - acc: 0.7787 - val_loss: 0.5411 - val_acc: 0.8125
Epoch 6/20
1360/1360 [==============================] - 0s 133us/step - loss: 0.4655 - acc: 0.7875 - val_loss: 0.5312 - val_acc: 0.8167
Epoch 7/20
1360/1360 [==============================] - 0s 131us/step - loss: 0.4643 - acc: 0.7794 - val_loss: 0.5243 - val_acc: 0.8083
Epo

Using a PCA doesn't seems to improve the accuracy, but the accuracy seems to be more stable with respect to the epochs